In [1]:
import numpy as np
import pandas as pd
import obspy as ob
import matplotlib.pyplot as plt
import math
from scipy import signal
from matplotlib import colormaps

from obspy.clients.fdsn.client import Client
from obspy.core.utcdatetime import UTCDateTime

from obspy.core.inventory import read_inventory
from obspy.core.inventory import read_inventory
from obspy.core import read

In [2]:
def linear_interpolation(trace,interpolation_limit=1):
    """Snippet to interpolate missing data.  

    The SHZ traces have missing data samples 3-4 times every 32 samples. 
    Providing the seed data with these missing data would mean using very 
    large files. Instead, we provide the data with -1 replacing the gaps. 
    To change the files to interpolate across the gaps, use this simple method to 
    replace the -1 values. The trace is modified, and a mask is applied at 
    the end if necessary. 

    :type stream: :class:`~obspy.core.Trace` 
    :param trace: A data trace
    :type interpolation_limit: int 
    :param interpolation_limit: Limit for interpolation. Defaults to 1. For
      more information read the options for the `~pandas.Series.interpolate`
      method. 

    :return: original_mask :class:`~numpy.ndarray` or class:`~numpy.bool_`
       Returns the original mask, before any interpolation is made. 

    """

    trace.data = np.ma.masked_where(trace.data == -1, trace.data)
    original_mask = np.ma.getmask(trace.data)
    data_series = pd.Series(trace.data)
    # data_series.replace(-1.0, pd.NA, inplace=True)
    data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)
    data_series.fillna(-1.0, inplace=True)
    trace.data=data_series.to_numpy(dtype=int) 
    trace.data = np.ma.masked_where(trace.data == -1, trace.data)
    return original_mask

In [3]:
def load_Apollo(starttime=None, endtime=None, station="A11", channel="SHZ", 
                units="DU", year=None, month=None):
    """
    Download seismogram stream for given parameters.
    
    If units=='DU' (default), returns raw digital units.
    If units=='ACC', instrument response is removed so that output is acceleration.
    
    You can specify either:
      1) starttime and endtime (as UTCDateTime objects), OR
      2) year and month (as integers) to retrieve the entire month.

    :param starttime: ObsPy UTCDateTime start
    :param endtime:   ObsPy UTCDateTime end
    :param station:   Station code (e.g., 'A11')
    :param channel:   Channel code (e.g., 'SHZ')
    :param units:     'DU' for raw digital units, or 'ACC' for acceleration
    :param year:      Optional integer specifying year if selecting entire month
    :param month:     Optional integer specifying month if selecting entire month
    :return:          Tuple of:
                      (Trace with mask applied, [starttime, endtime, station, channel])
    """
    import numpy as np
    import pandas as pd
    from obspy import UTCDateTime
    from obspy.clients.fdsn import Client
    
    # --- 1. Figure out time window ---
    if (starttime is not None or endtime is not None) and (year is not None or month is not None):
        raise ValueError("Please specify either (starttime, endtime) OR (year, month), not both.")
    
    if year is not None and month is not None:
        # Create a starttime as the first day of the given month
        starttime = UTCDateTime(year, month, 1, 0, 0, 0)
        # Create an endtime as the last second of that same month
        if month == 12:
            endtime = UTCDateTime(year + 1, 1, 1, 0, 0, 0) - 1
        else:
            endtime = UTCDateTime(year, month + 1, 1, 0, 0, 0) - 1
    else:
        # If no year/month was given, we assume starttime/endtime are valid
        if starttime is None or endtime is None:
            raise ValueError("You must provide either (starttime, endtime) OR (year, month).")


    # --- 2. Use FDSN client to get data and process ---
    client = Client("IRIS")

    # Retrieve station metadata (inventory) for response
    inv = client.get_stations(starttime=starttime, endtime=endtime,
                              network='XA', sta=station, loc='*', channel=channel,
                              level="response")
    
    # Retrieve waveforms
    st = client.get_waveforms(network='XA', station=station, channel=channel, location='*',
                              starttime=starttime, endtime=endtime)
    st.merge()
    if not st:
             print(f"Warning: No data returned via FDSN for {station} {channel} between {starttime} and {endtime}")
             return None, [starttime, endtime, station, channel]
    for tr in st:
        if tr.data is not None and len(tr.data) > 0 and np.issubdtype(tr.data.dtype, np.number):
            try:
                # Calculate the median value of the trace data
                # Use np.ma.median if data might already contain masks/NaNs after merging
                if isinstance(tr.data, np.ma.MaskedArray):
                    # Calculate median ignoring masked values
                    median_val = np.ma.median(tr.data)
                else:
                    # Calculate median on the numpy array
                    median_val = np.median(tr.data)

                # Check if median calculation was successful (e.g., not NaN)
                if median_val is not None and not np.isnan(median_val):
                    invalid_value = median_val - 511

                    # Create a boolean mask where data equals the invalid value
                    invalid_mask = (tr.data == invalid_value)

                    # Apply the mask if any invalid values were found
                    if np.any(invalid_mask):
                        # Ensure the data is a masked array before applying the mask
                        if not isinstance(tr.data, np.ma.MaskedArray):
                             tr.data = np.ma.masked_array(tr.data, mask=invalid_mask)
                        else:
                             # Combine with existing mask using logical OR
                             tr.data.mask = np.logical_or(tr.data.mask, invalid_mask)
                        # Optional: print message
                        # num_masked = np.sum(invalid_mask)
                        # print(f"Masked {num_masked} points with invalid value {invalid_value} in trace {tr.id}.")

                # else: # Optional: handle cases where median couldn't be calculated
                #     print(f"Warning: Could not calculate a valid median for trace {tr.id}. Skipping invalid value masking.")

            except Exception as e:
                # Keep the original trace data if masking fails
                print(f"Warning: Error during invalid value masking for trace {tr.id}: {e}. Proceeding with original data.")
    

    # Process gaps with linear interpolation
    for tr in st:
        linear_interpolation(tr, interpolation_limit=1)

    # Remove instrument response if needed
    for tr in st:
        # On Apollo missions, channels can be MH1, MH2, MHZ, or SHZ, etc.
        if tr.stats.channel in ['MH1', 'MH2', 'MHZ']:
            original_mask = linear_interpolation(tr, interpolation_limit=None)
            if units.upper() == 'DISP':
                # Pre-filter band might be different for MH* channels
                pre_filt = [1/350, 1/300, 1.5, 2]
                tr.remove_response(inventory=inv, pre_filt=pre_filt,
                                   output="DISP", water_level=None, plot=False)
            tr.data = np.ma.masked_array(tr.data, mask=original_mask)
        elif tr.stats.channel in ['SHZ']:
            original_mask = linear_interpolation(tr, interpolation_limit=None)
            if units.upper() == 'DISP':
                # Different pre-filter band for SHZ
                pre_filt = [1/100, 1/50, 20, 22]
                tr.remove_response(inventory=inv, pre_filt=pre_filt,
                                   output="DISP", water_level=None, plot=False)
            tr.data = np.ma.masked_array(tr.data, mask=original_mask)

    # For simplicity, return just the first Trace in the Stream
    return st[0], [starttime, endtime, station, channel]

In [12]:
import gc
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client

# -----------------------------
# Helper function: preprocess_trace
def preprocess_trace(trace):
    """
    Pre-processing workflow:
      1. Fill masked data with interpolation if possible
      2. Detrend (linear) and demean
      3. Apply de-glitching
    """
    proc = trace.copy()
    
    # Handle masked data
    if np.ma.is_masked(proc.data):
        mask = np.ma.getmaskarray(proc.data)
        if np.all(mask):
            proc.data = np.zeros_like(proc.data, dtype=float)
        else:
            x = np.arange(len(proc.data))
            valid_indices = ~mask
            if np.any(valid_indices):
                valid_x = x[valid_indices]
                valid_y = proc.data[valid_indices]
                proc.data = np.interp(x, valid_x, valid_y)
            else:
                proc.data = proc.data.filled(0)
    
    # Detrend and demean
    proc.detrend(type='linear')
    proc.detrend(type='demean')
    
    # Apply de-glitching
    #proc.data = deglitch_data(proc.data, threshold=3)
    
    return proc

# -----------------------------
# Function: Process Event
def process_event(event_row, avail_map):
    # Parse event time
    try:
        event_date_str = event_row["Event Date"]
        year_str, month_str, day_time = event_date_str.split('/', 2)
        day_str, time_str = day_time.split(' ', 1)
        full_year = 1900 + int(year_str)
        starttime = UTCDateTime(f"{full_year}-{month_str.zfill(2)}-{day_str.zfill(2)}T{time_str}:00")
        stop_time_str = str(int(event_row["Stop Time\n(HHMM)"])).zfill(4)
        stop_hour = stop_time_str[:2]
        stop_min = stop_time_str[2:]
        endtime = UTCDateTime(f"{full_year}-{month_str.zfill(2)}-{day_str.zfill(2)}T{stop_hour}:{stop_min}:00")
    except Exception as e:
        print(f"Error parsing event time for event {event_row['No']}: {e}")
        return

    # Notice: change event duration here if needed
    event_time_str = starttime.strftime("%Y%m%dT%H%M%S")
    extended_start = starttime - 900  # 15 minutes before start
    extended_end = endtime + 1800    # 30 minutes after stop

    # Sensor-channel definitions
    sensors_channels = {
        "LP": ["MHZ"], # Notice: change here if you need other components, for example, "LP": ["MH1", "MH2", "MHZ"],
        "SP": ["SHZ"]
    }

    for station, col_index in avail_map.items():
        if pd.notna(event_row.iloc[col_index]):
            print(f"Event {event_row['No']} available at station {station}.")
            
            # Process LP and SP
            for sensor, channels in sensors_channels.items():
                for chan in channels:
                    try:
                        event_trace, _ = load_Apollo(starttime=extended_start, endtime=extended_end,
                                                     station=station, channel=chan, units="DU")
                        if event_trace is None or event_trace.data is None or len(event_trace.data) == 0:
                            print(f"[{station} {chan} {sensor}] No event data – skipping.")
                            continue
                        
                        # Preprocess the trace
                        event_trace = preprocess_trace(event_trace)
                        
                        # Create output folder
                        output_folder = os.path.join("Impact_test", station, sensor, event_time_str, chan)
                        os.makedirs(output_folder, exist_ok=True)
                        
                        # Save seismogram text
                        seismogram_txt = os.path.join(output_folder, "seismogram.txt")
                        time_datetimes = [(event_trace.stats.starttime + i * event_trace.stats.delta).datetime 
                                         for i in range(len(event_trace.data))]
                        time_strings = [t.isoformat() for t in time_datetimes]
                        with open(seismogram_txt, "w") as f:
                            f.write("Time Amplitude\n")
                            for ts, amplitude in zip(time_strings, event_trace.data):
                                f.write(f"{ts} {amplitude:f}\n")
                        
                        # Save seismogram plot
                        plt.figure()
                        plt.plot(time_datetimes, event_trace.data)
                        plt.xlabel("Time")
                        plt.ylabel("Amplitude")
                        plt.title(f"Seismogram for {station} {chan} ({sensor})")
                        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
                        plt.gcf().autofmt_xdate()
                        seismogram_png = os.path.join(output_folder, "seismogram.png")
                        plt.savefig(seismogram_png)
                        plt.close()
                        
                        print(f"Event {event_row['No']}, Station {station}, Channel {chan} ({sensor}): Saved seismogram.")
                        
                        # Clean up
                        del event_trace
                        gc.collect()
                    except Exception as e:
                        print(f"Error processing station {station} channel {chan} ({sensor}): {e}")
                        continue
        else:
            print(f"Event {event_row['No']} not available at station {station}.")

    with open("checkpoint_impact_test.txt", "a") as f:
        f.write(f"Processed event {event_row['No']} at {UTCDateTime.now()}\n")
    gc.collect()

# -----------------------------
# Main Function
def main():
    csv_file = "Nakamura_MQCatalog.levent.1008 16.21.csv"
    try:
        df = pd.read_csv(csv_file)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    print("CSV Columns:")
    print(df.columns.tolist())

    # Filter 
    cols = df.columns.tolist()
    # For Shallow moonquakes and Impacts, use the following
    try:
        idx_et = cols.index("Event Type")
    except Exception as e:
        print("Error: 'Event Type' column not found.", e)
        return
    filtered_events = df[(df["Event Type"].str.strip() == "C")] #For meteoriod impacts
    #filtered_events = df[(df["Event Type"].str.strip() == "H")] #For shallow moonquakes
    print(f"Found {len(filtered_events)} impacts.")

    # For Deep moonquakes, use the following

    #et_col   = next(c for c in cols if "New DMQ" in c)
    #idx_et   = cols.index(et_col)
    #classifier_col = cols[idx_et+1]
    #df[classifier_col] = pd.to_numeric(
    #    df[classifier_col].astype(str).str.strip(),
    #    errors='coerce'
    #)
    #filt = df[(df[et_col].str.strip()=="A") & (df[classifier_col]==7)]
    # Notice: if impact
    # filt = df[(df[et_col].str.strip()=="A")
    #print(f"{len(filt)} A7 events")

    # Availability mapping, uncomment if you need other stations
    availability_map = {
        #"S12": cols.index("Availability"),
        #"S14": cols.index("Availability") + 1,
        "S15": cols.index("Availability") + 2,
        "S16": cols.index("Availability") + 3
    }
    print("Availability mapping (station: column index):")
    print(availability_map)

    # Checkpoint handling
    checkpoint_file = "checkpoint_impact_test.txt"
    processed_events = set()
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r") as f:
            for line in f:
                if line.startswith("Processed event"):
                    parts = line.split()
                    event_no = parts[2]
                    processed_events.add(event_no)

    # Process events
    for idx, event_row in filtered_events.iterrows():
        if str(event_row["No"]) in processed_events:
            print(f"Skipping event {event_row['No']} (already processed).")
            continue
        process_event(event_row, availability_map)
        processed_events.add(str(event_row['No']))
        gc.collect()

    # Print folder structure
    print("Final folder structure (relative paths):")
    for root, dirs, files in os.walk("Impact_test"):
        level = root.replace("Impact_test", "").count(os.sep)
        indent = " " * 4 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = " " * 4 * (level + 1)
        for f in files:
            print(f"{subindent}{f}")

if __name__ == "__main__":
    main()

CSV Columns:
['No', 'Year', 'Day of the year', 'Month', 'Day', 'Start Time\n(HHMM)', 'Stop Time\n(HHMM)', 'Start Time\n(HHMM).1', 'Stop Time\n(HHMM).1', 'Event Date', '30 minutes\nBefore event', '3 Hours\nAfter Event', '30 minutes\nBefore event.1', '3 Hours\nAfter Event.1', 'Signal envelope amplitudes', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Availability', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Data quality code', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Comment', 'Event Type', 'Unnamed: 28', 'New DMQ', 'Unnamed: 30', 'Article Usage', 'Unnamed: 32']
Found 1744 impacts.
Availability mapping (station: column index):
{'S15': 20, 'S16': 21}
Event 2.0 not available at station S15.
Event 2.0 not available at station S16.
Event 4.0 not available at station S15.
Event 4.0 not available at station S16.
Event 5.0 not available at station S15.
Event 5.0 not available at station S16.
Event 6.0 not available at station S15.
Event 6.0 not available at station S16.
Event 14.0 not a

/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event 1631.0, Station S15, Channel MHZ (LP): Saved seismogram.


/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event 1631.0, Station S15, Channel SHZ (SP): Saved seismogram.
Event 1631.0 not available at station S16.
Event 1634.0 not available at station S15.
Event 1634.0 not available at station S16.
Event 1636.0 not available at station S15.
Event 1636.0 not available at station S16.
Event 1644.0 available at station S15.


...

/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event 1762.0, Station S15, Channel SHZ (SP): Saved seismogram.
Event 1762.0 not available at station S16.
Event 1767.0 available at station S15.


/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event 1767.0, Station S15, Channel MHZ (LP): Saved seismogram.


KeyboardInterrupt: 

In [14]:
## shallow moonquakes onodera2024
# SP SHZ only

import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from datetime import datetime, timedelta

# Helper function: preprocess_trace (unchanged)
def preprocess_trace(trace):
    """
    Pre-processing workflow:
      1. Fill masked data with interpolation if possible
      2. Detrend (linear) and demean
      3. Apply de-glitching
    """
    proc = trace.copy()
    
    if np.ma.is_masked(proc.data):
        mask = np.ma.getmaskarray(proc.data)
        if np.all(mask):
            proc.data = np.zeros_like(proc.data, dtype=float)
        else:
            x = np.arange(len(proc.data))
            valid_indices = ~mask
            if np.any(valid_indices):
                valid_x = x[valid_indices]
                valid_y = proc.data[valid_indices]
                proc.data = np.interp(x, valid_x, valid_y)
            else:
                proc.data = proc.data.filled(0)
    
    proc.detrend(type='linear')
    proc.detrend(type='demean')
    
    #proc.data = deglitch_data(proc.data, threshold=3)
    
    return proc

# Helper function: parse_stations
def parse_stations(station_str):
    """
    Parse the Station column to extract a list of stations.
    Format examples: 'S15', 'S14 (+S15)', 'S14 (+S15, S16)'
    """
    if "(" in station_str:
        main_station, additional = station_str.split(" (+")
        main_station = main_station.strip()
        additional = additional.replace(")", "").split(",")
        additional_stations = [s.strip() for s in additional]
        stations = [main_station] + additional_stations
    else:
        stations = [station_str.strip()]
    return stations

# Function: Process Event
def process_event(event_row):
    # Parse event time from Year, DOY, and Start time (UTC)
    try:
        year = int(event_row["Year"])
        doy = int(event_row["DOY"])
        start_time_str = event_row["Start time (UTC)"]
        # Convert DOY to date
        date = datetime(year, 1, 1) + timedelta(days=doy - 1)
        # Parse start time (HH:MM:SS)
        hour, minute, second = map(int, start_time_str.split(':'))
        start_datetime = datetime(date.year, date.month, date.day, hour, minute, second)
        starttime = UTCDateTime(start_datetime)
        endtime = starttime + 1800  # 30 minutes after start
    except Exception as e:
        print(f"Error parsing event time for event {event_row['Event ID']}: {e}")
        return

    event_time_str = starttime.strftime("%Y%m%dT%H%M%S")
    
    # Parse stations from the Station column
    station_str = event_row["Station"]
    stations = parse_stations(station_str)
    
    # Process only SP SHZ data for each station
    for station in stations:
        try:
            event_trace, _ = load_Apollo(starttime=starttime, endtime=endtime,
                                         station=station, channel="SHZ", units="DU")
            if event_trace is None or event_trace.data is None or len(event_trace.data) == 0:
                print(f"[{station} SHZ SP] No event data – skipping.")
                continue
            
            # Preprocess the trace
            event_trace = preprocess_trace(event_trace)
            
            # Create output folder
            output_folder = os.path.join("Shallow_test", station, "SP", event_time_str, "SHZ")
            os.makedirs(output_folder, exist_ok=True)
            
            # Save seismogram text
            seismogram_txt = os.path.join(output_folder, "seismogram.txt")
            time_datetimes = [(event_trace.stats.starttime + i * event_trace.stats.delta).datetime 
                              for i in range(len(event_trace.data))]
            time_strings = [t.isoformat() for t in time_datetimes]
            with open(seismogram_txt, "w") as f:
                f.write("Time Amplitude\n")
                for ts, amplitude in zip(time_strings, event_trace.data):
                    f.write(f"{ts} {amplitude:f}\n")
            
            # Save seismogram plot
            plt.figure()
            plt.plot(time_datetimes, event_trace.data)
            plt.xlabel("Time")
            plt.ylabel("Amplitude")
            plt.title(f"Seismogram for {station} SHZ (SP)")
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
            plt.gcf().autofmt_xdate()
            seismogram_png = os.path.join(output_folder, "seismogram.png")
            plt.savefig(seismogram_png)
            plt.close()
            
            print(f"Event {event_row['Event ID']}, Station {station}, Channel SHZ (SP): Saved seismogram.")
            
            # Clean up
            del event_trace
            gc.collect()
        except Exception as e:
            print(f"Error processing station {station} channel SHZ (SP): {e}")
            continue

    # Update checkpoint
    with open("checkpoint_shallow_test.txt", "a") as f:
        f.write(f"Processed event {event_row['Event ID']} at {UTCDateTime.now()}\n")
    gc.collect()

# Main Function
def main():
    csv_file = "Onodera2024_TableA1_New_Shallow_Moonquakes_Complete.csv"
    try:
        df = pd.read_csv(csv_file)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    print("CSV Columns:")
    print(df.columns.tolist())

    # Checkpoint handling
    checkpoint_file = "checkpoint_shallow_test.txt"
    processed_events = set()
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r") as f:
            for line in f:
                if line.startswith("Processed event"):
                    parts = line.split()
                    event_id = parts[2]
                    processed_events.add(event_id)

    # Process all events (no filtering needed as all are shallow moonquakes)
    for idx, event_row in df.iterrows():
        event_id = event_row["Event ID"]
        if event_id in processed_events:
            print(f"Skipping event {event_id} (already processed).")
            continue
        process_event(event_row)
        processed_events.add(event_id)
        gc.collect()

    # Print folder structure
    print("Final folder structure (relative paths):")
    for root, dirs, files in os.walk("Shallow_test"):
        level = root.replace("Shallow_test", "").count(os.sep)
        indent = " " * 4 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = " " * 4 * (level + 1)
        for f in files:
            print(f"{subindent}{f}")

if __name__ == "__main__":
    main()

CSV Columns:
['Event ID', 'Year', 'DOY', 'Start time (UTC)', 'Ts–Tp (s)', 'Hypocentral distance (km)', 'Min. fc (Hz)', 'Min. Erel (J)', 'Min. M0 (Nm)', 'Min. Δσ (MPa)', 'Min. mb', 'Station']


/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event KO‐SMQ‐1, Station S15, Channel SHZ (SP): Saved seismogram.


/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event KO‐SMQ‐2, Station S14, Channel SHZ (SP): Saved seismogram.


...

/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event KO‐SMQ‐4, Station S14, Channel SHZ (SP): Saved seismogram.


/var/folders/6c/rybd089s7ssd_ybsnsgsxvd40000gp/T/ipykernel_39066/2006576976.py:27: FutureWarning: The 'downcast' keyword in Series.interpolate is deprecated and will be removed in a future version. Call result.infer_objects(copy=False) on the result instead.
  data_series.interpolate(method='linear', axis=0, limit=interpolation_limit, inplace=True, limit_direction=None, limit_area='inside', downcast=None)


Event KO‐SMQ‐4, Station S15, Channel SHZ (SP): Saved seismogram.


KeyboardInterrupt: 